### Get this to work for new version of fastai
We need to load a pth torch object instead of fastai learner (from pkl).

This is because pickle preserve of the python object breaks when loading with new import name.


10.14.2020

In [1]:
import torch
from fastai.vision.all import *

In [2]:
!ls ../models -sh

total 1.6G
 99M base-learner-12.pkl	   0 model-config    105M myproto-8.pkl
 99M base-learner-13-fit25.pkl	   0 model-logs      105M myproto-9.pkl
107M base-learner-13-fit25.pth	105M myproto-0.pkl   105M myproto_valid-0.pkl
 99M base-learner-6.pkl		105M myproto-10.pkl   99M resnet50-acc75.pkl
 48M bing-clean-2.pkl		105M myproto-1.pkl    53M stadard-piece-2.pkl
 46M chess1.pkl			   0 myproto-3.pkl      0 tmpmod-a-0.pkl
101M expmod-b-1.pkl		105M myproto-5.pkl    51M tmpmod-a-1.pkl
   0 model-analysis		105M myproto-7.pkl


In [5]:
def piece_class_parse(fn): 
    fn = fn.split('_')[1]
    fn = fn.split('.')[0]
    return fn

### Build Learn + Train_DL

In [7]:
_train_path = Path('../../../rf-chess-data/cropped_v1/')

In [8]:
_train_path.ls()

(#2871) [Path('../../../rf-chess-data/cropped_v1/00000_white-pawn.jpg'),Path('../../../rf-chess-data/cropped_v1/00001_black-queen.jpg'),Path('../../../rf-chess-data/cropped_v1/00002_black-pawn.jpg'),Path('../../../rf-chess-data/cropped_v1/00003_white-pawn.jpg'),Path('../../../rf-chess-data/cropped_v1/00004_white-bishop.jpg'),Path('../../../rf-chess-data/cropped_v1/00005_white-king.jpg'),Path('../../../rf-chess-data/cropped_v1/00006_white-queen.jpg'),Path('../../../rf-chess-data/cropped_v1/00007_white-rook.jpg'),Path('../../../rf-chess-data/cropped_v1/00008_white-knight.jpg'),Path('../../../rf-chess-data/cropped_v1/00009_white-pawn.jpg')...]

In [9]:
train_fnames = get_image_files(_train_path)

In [10]:
train_dl = ImageDataLoaders.from_name_func(
                    _train_path, 
                    train_fnames,
                    valid_pct=.2, 
#                     seed=_train_seed,
                    label_func=piece_class_parse, 
#                     item_tfms=Crop,
#                     batch_tfms=Augs,
#                     bs=_bs,
                    # num_workers=0,
                    )

In [11]:
train_dl

In [12]:
learn = cnn_learner(train_dl, resnet50, metrics=[],
                        normalize=False)


### Load StateDict from pth

In [14]:
PATH = '../models/base-learner-13-fit25.pth'

model = learn.model

model.load_state_dict(torch.load(PATH))

model.eval()

### View Predictions - not great

In [20]:
import os
d = '../../../other-chess-data/regulation-pieces-1//crops/'
fns = os.listdir(d)
img_fns = [d + fn for fn in fns[:10]]


In [23]:
actuals = [Path(e).name for e in img_fns]

In [24]:
preds = [learn.predict(e)[0] for e in img_fns]

In [25]:
[(a,b) for a,b in zip(actuals, preds)]

[('00000_white-king.jpg', 'white-knight'),
 ('00001_black-rook.jpg', 'black-rook'),
 ('00002_black-knight.jpg', 'black-rook'),
 ('00003_white-knight.jpg', 'white-pawn'),
 ('00004_white-pawn.jpg', 'white-pawn'),
 ('00005_black-pawn.jpg', 'white-bishop'),
 ('00006_white-bishop.jpg', 'white-pawn'),
 ('00007_white-rook.jpg', 'white-pawn'),
 ('00008_white-queen.jpg', 'white-knight'),
 ('00009_black-bishop.jpg', 'white-pawn')]